In [5]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
from pprint import pprint
import pandas as pd
import numpy as np
from citipy import citipy
from scipy.stats import sem

In [6]:
#Dependencies to merge the csv's
#Import packages
import os
import glob
import csv

In [7]:
# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Files to Load
#File for 2019
csv_2019_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2019.csv")
#File for 2018
csv_2018_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2018.csv")
#File for 2017
csv_2017_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2017.csv")
#File for 2016
csv_2016_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2016.csv")
#File for 2015
csv_2015_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2015.csv")
#File for 2014
csv_2014_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2014.csv")
#File for 2013
csv_2013_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2013.csv")
#File for 2012
csv_2012_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2012.csv")
#File for 2011
csv_2011_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2011.csv")
#File for 2010
csv_2010_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2010.csv")
#File for 2009
csv_2009_df=pd.read_csv("../Project1_DC_Crime_Analysis/Resources/Crime_Incidents_in_2009.csv")

In [9]:
#Count the number of rows in each file
#csv_2019_df["REPORT_DAT"].count() #27145
#csv_2018_df["REPORT_DAT"].count() #33783
#csv_2017_df["REPORT_DAT"].count() #33116
#csv_2016_df["REPORT_DAT"].count() #37227
#csv_2015_df["REPORT_DAT"].count() #37328
#csv_2014_df["REPORT_DAT"].count() #38438
#csv_2013_df["REPORT_DAT"].count() #35895
#csv_2012_df["REPORT_DAT"].count() #35317
#csv_2011_df["REPORT_DAT"].count() #33292
#csv_2010_df["REPORT_DAT"].count() #31676
#csv_2009_df["REPORT_DAT"].count() #31312

In [10]:
#Use glob to match the pattern ‘csv’
#Match the pattern (‘csv’) and save the list of file names in the ‘all_files’ variable.
#extension = 'csv'
#Use pandas to concatenate all files in the list and export as CSV. 
#The output file is named “dc_crimes.csv” located in your working directory.
#combine all files in the list
path = '../Project1_DC_Crime_Analysis/Resources/'
all_files = glob.glob(path + "/*.csv")
all_files
row = []

for filename in all_files:
   df = pd.read_csv(filename, index_col=None, header=0)
   row.append(df)

dc_crimes = pd.concat(row, axis=0, ignore_index=True)
dc_crimes_df=pd.DataFrame(dc_crimes)
dc_crimes_df.count()
dc_crimes_df

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.039673,38.903151,8005297,2009-01-12T10:30:00.000Z,DAY,OTHERS,BURGLARY,1000 - 1099 BLOCK OF CONNECTICUT AVENUE NW,396559.0,137255.0,...,010700 1,10700.0,Precinct 17,38.903143,-77.039670,GOLDEN TRIANGLE,2008-12-28T00:00:00.000Z,NaN,356048194,08005297-01
1,-76.999518,38.904689,162302,2009-11-12T18:00:00.000Z,EVENING,OTHERS,MOTOR VEHICLE THEFT,1100 - 1199 BLOCK OF 5TH STREET NE,400042.0,137425.0,...,010600 2,10600.0,Precinct 83,38.904682,-76.999516,NaN,2009-11-06T12:00:00.000Z,2009-11-12T17:00:00.000Z,356046669,00162302-01
2,-77.022949,38.917236,174072,2009-12-06T23:30:00.000Z,MIDNIGHT,OTHERS,THEFT F/AUTO,2000 - 2099 BLOCK OF 8TH STREET NW,398010.0,138818.0,...,003500 2,3500.0,Precinct 37,38.917228,-77.022947,NaN,2009-12-06T21:30:00.000Z,2009-12-06T23:30:00.000Z,356046670,00174072-01
3,-77.021912,38.919497,2001387,2009-10-20T08:59:00.000Z,DAY,OTHERS,THEFT/OTHER,2200 - 2299 BLOCK OF GEORGIA AVENUE NW,398100.0,139069.0,...,003500 2,3500.0,Precinct 37,38.919489,-77.021910,NaN,2002-01-26T17:00:00.000Z,2002-01-26T00:00:00.000Z,356046672,02001387-01
4,-77.021920,38.907912,5147017,2009-10-20T09:32:00.000Z,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,1300 - 1399 BLOCK OF 7TH STREET NW,398099.0,137783.0,...,004902 1,4902.0,Precinct 18,38.907905,-77.021918,NaN,2005-04-30T06:45:00.000Z,2005-04-30T00:00:00.000Z,356046677,05147017-01
5,-76.965694,38.853003,6165160,2009-11-18T10:44:00.000Z,DAY,OTHERS,MOTOR VEHICLE THEFT,2966 - 3031 BLOCK OF 30TH STREET SE,402978.0,131688.0,...,007502 2,7502.0,Precinct 115,38.852995,-76.965692,NaN,2009-11-16T17:30:00.000Z,2009-11-18T08:45:00.000Z,356046683,06165160-01
6,-77.009722,38.824388,7127054,2009-09-05T01:55:00.000Z,MIDNIGHT,OTHERS,THEFT/OTHER,2 - 153 BLOCK OF GALVESTON PLACE SW,399156.0,128511.0,...,009807 2,9807.0,Precinct 126,38.824380,-77.009719,NaN,2009-09-05T01:43:00.000Z,NaN,356046689,07127054-01
7,-77.022020,38.932983,7140698,2009-02-26T00:00:00.000Z,MIDNIGHT,OTHERS,HOMICIDE,500 - 699 BLOCK OF PARK ROAD NW,398091.0,140566.0,...,003200 3,3200.0,Precinct 38,38.932975,-77.022017,NaN,2007-10-13T01:40:00.000Z,2007-10-13T00:00:00.000Z,356046691,07140698-01
8,-77.014225,38.943803,8092867,2009-02-22T00:00:00.000Z,MIDNIGHT,OTHERS,HOMICIDE,4300 - 4399 BLOCK OF 2ND STREET NW,398767.0,141767.0,...,002301 2,2301.0,Precinct 46,38.943795,-77.014223,NaN,2008-07-05T15:42:00.000Z,2008-07-05T00:00:00.000Z,356063677,08092867-01
9,-76.947655,38.896660,8106174,2009-09-01T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,4000 - 4121 BLOCK OF MINNESOTA AVENUE NE,404541.0,136535.0,...,007803 1,7803.0,Precinct 99,38.896652,-76.947653,NaN,2008-07-29T18:29:00.000Z,2008-07-29T18:30:00.000Z,356066228,08106174-01


In [11]:
#Split the data in REPORT_DAT column to get Date and Time information
dc_crimes_df[['REPORT_DAT','Time']] = dc_crimes_df['REPORT_DAT'].str.split('T',expand=True)

In [12]:
#Display the data
dc_crimes_df.head(2)

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,Time
0,-77.039673,38.903151,8005297,2009-01-12,DAY,OTHERS,BURGLARY,1000 - 1099 BLOCK OF CONNECTICUT AVENUE NW,396559.0,137255.0,...,10700.0,Precinct 17,38.903143,-77.039670,GOLDEN TRIANGLE,2008-12-28T00:00:00.000Z,NaN,356048194,08005297-01,10:30:00.000Z
1,-76.999518,38.904689,162302,2009-11-12,EVENING,OTHERS,MOTOR VEHICLE THEFT,1100 - 1199 BLOCK OF 5TH STREET NE,400042.0,137425.0,...,10600.0,Precinct 83,38.904682,-76.999516,NaN,2009-11-06T12:00:00.000Z,2009-11-12T17:00:00.000Z,356046669,00162302-01,18:00:00.000Z


In [13]:
#Counts
dc_crimes_df.count()

X                       374403
Y                       374403
CCN                     374403
REPORT_DAT              374403
SHIFT                   374403
METHOD                  374403
OFFENSE                 374403
BLOCK                   374402
XBLOCK                  374403
YBLOCK                  374403
WARD                    374392
ANC                     374403
DISTRICT                374201
PSA                     374173
NEIGHBORHOOD_CLUSTER    369548
BLOCK_GROUP             373335
CENSUS_TRACT            373335
VOTING_PRECINCT         374339
LATITUDE                374403
LONGITUDE               374403
BID                      62208
START_DATE              374393
END_DATE                352236
OBJECTID                374403
OCTO_RECORD_ID          374403
Time                    374403
dtype: int64

In [14]:
#Determine Day of the week and Week number
#Use the to_datetime function of Pandas
dc_crimes_df['REPORT_DAT'] = pd.to_datetime(dc_crimes_df['REPORT_DAT'])
dc_crimes_df['day_of_week'] = dc_crimes_df['REPORT_DAT'].dt.day_name()
dc_crimes_df['Week_Number'] = dc_crimes_df['REPORT_DAT'].dt.week
dc_crimes_df[["REPORT_DAT","day_of_week","Week_Number"]]

,REPORT_DAT,day_of_week,Week_Number
0,2009-01-12,Monday,3
1,2009-11-12,Thursday,46
2,2009-12-06,Sunday,49
3,2009-10-20,Tuesday,43
4,2009-10-20,Tuesday,43
5,2009-11-18,Wednesday,47
6,2009-09-05,Saturday,36
7,2009-02-26,Thursday,9
8,2009-02-22,Sunday,8
9,2009-09-01,Tuesday,36


In [15]:
# Retrieving post codes from Latitudes and Longitudes
dc_crimes_df[["LATITUDE","LONGITUDE"]].head(5)

,LATITUDE,LONGITUDE
0,38.903143,-77.039670
1,38.904682,-76.999516
2,38.917228,-77.022947
3,38.919489,-77.021910
4,38.907905,-77.021918


In [ ]:
plt.scatter(dc_crimes_df['LONGITUDE'], dc_crimes_df['LATITUDE'], s=50, alpha=0.3, color="brown", lw=0)

In [ ]:
#Export the city data into a .csv. -"Should be the final step"
#dc_crimes_df.to_csv("../Project1-Additional_Docs/Analyzed_outputs/dc_crimes.csv",index=False,header=True,encoding='utf-8-sig')
#encoding also exports any non-english data